# s1 backscatter for lake ice cover

In [1]:
# use `pro` env
from dask.distributed import LocalCluster
from functools import reduce
import geopandas as gpd
from glob import glob
import numpy as np
import logging
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import pandas as pd
import rioxarray as rio
import seaborn as sns
import stackstac
import xarray as xr
import lakeIce_utils as liU
from tqdm import tqdm
from scipy.stats import linregress

import importlib
importlib.reload(liU)

<module 'lakeIce_utils' from '/scratch/s1759665/paper2/src/lakeIce_utils.py'>

In [2]:
centrelines = gpd.read_file('../data/streams_v3.geojson')

basins = (gpd.read_file('../data/basins/Greenland_Basins_PS_v1.4.2.shp')
          .dissolve('SUBREGION1'))
centrelines = centrelines.sjoin(basins.drop(columns=['NAME', 'GL_TYPE']),
                                how='left'
                                ).rename(columns={'index_right': 'region'})

lakes = gpd.read_file('../data/lake_areas.geojson')

lakes = lakes.sjoin_nearest(centrelines)
lakes = (lakes
            .sort_values(by='id')
            .drop(columns=['index_right', 'name', 'lake_land'])
            .to_crs(4326)
)

In [3]:
with LocalCluster(n_workers=8,
                  threads_per_worker=2,
                  memory_limit='1G',
                  silence_logs=logging.ERROR) as cluster:
    
    client = cluster.get_client()
    print(client.dashboard_link)
    
    li = liU.Sentinel1(lakes.loc[1], export=True, sample=False)


http://127.0.0.1:8787/status


/home/s1759665/micromamba/envs/pro/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/s1759665/micromamba/envs/pro/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/s1759665/micromamba/envs/pro/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/s1759665/micromamba/envs/pro/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/s1759665/micromamba/envs/pro/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/s1759665/micromamba/envs/pro/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encounter

already got incident angle ds...reading from .zarr


In [ ]:
li.dBq

In [ ]:
attrs = li.dBq.attrs
attrs['id'] = str(attrs['id'])
li.dBq.attrs = attrs

In [ ]:
li.dBq.to_parquet(li.quantile_export_path)

In [ ]:
df = pd.concat([pd.read_parquet(f) for f in glob('../results/lakeIce/*_sample.parquet')])
df = df.melt(['xy', 'time', 'sat:relative_orbit', 'angle', 'month', 'id', 'region'])
df = df.rename(columns={
    'variable': 'band',
    'value': 'dB'
})
df['band'] = df['band'].str.replace('dB_', '')
df['angle'] = np.degrees(df['angle'])

In [ ]:
# get outliers (5th and 95th percentile) and ditch 'em before fitting model
outliers = (df
 .groupby('band')
 .apply(lambda q: np.quantile(q['dB'], [0.05, 0.95]))
 .apply(pd.Series)
 .rename(columns={0: 'q02', 1: 'q98'})
)

idxs = []
for band in outliers.index:
    idx = (
        (df['band'] == band)
        &
        (df['dB'] > outliers.loc[band, 'q02'])
        &
        (df['dB'] < outliers.loc[band, 'q98'])
    )
    idxs.append(idx)
    
idx = np.logical_or(*idxs)

In [ ]:
## backscatter dependence on angle by band and month
lr = (df[idx].groupby(['band', 'month'])[['angle', 'dB']].apply(lambda q: pd.Series(linregress(x = q['angle'],
                                                              y = q['dB'])))
      .reset_index()
      .rename(columns = {0:'slope',
                         1:'intercept',
                         2:'rvalue',
                         3:'pvalue',
                         4:'stderr'}))


minang, maxang = df['angle'].agg(['min', 'max'])
incident_angles = np.arange(minang//1, 1 + maxang//1)
month_norm = Normalize(1, 12)

print(lr.loc[
    lr['slope'].abs().sort_values(ascending=False).index
    ].to_markdown())

band_style = {
    'hh': ':',
    'hv': '-'
}
fig, ax = plt.subplots()

for fit in lr.itertuples():
    if fit.month == 6:
        label = fit.band
    else:
        label = None
    ax.plot(incident_angles,
            incident_angles * fit.slope + fit.intercept,
            c=plt.get_cmap('twilight')(month_norm(fit.month)),
            ls=band_style[fit.band],
            label=label)

ax.legend()

sm = ScalarMappable(month_norm, cmap='twilight')
plt.colorbar(sm, ax=ax, label='month')

ax.set(xlabel='Incidence Angle (degrees)',
       ylabel='Backscatter (dB)')

In [ ]:
## backscatter dependence on angle by BAND, only

lr = (df[idx].groupby(['band'])[['angle', 'dB']].apply(lambda q: pd.Series(linregress(x = q['angle'],
                                                              y = q['dB'])))
      .reset_index()
      .rename(columns = {0:'slope',
                         1:'intercept',
                         2:'rvalue',
                         3:'pvalue',
                         4:'stderr'}))

print(lr.to_markdown())

minang, maxang = df['angle'].agg(['min', 'max'])
incident_angles = np.arange(minang//1, 1 + maxang//1)

band_style = {
    'hh': ':',
    'hv': '-'
}
fig, ax = plt.subplots()
for fit in lr.itertuples():    
    ax.plot(incident_angles,
            incident_angles * fit.slope + fit.intercept,
            ls=band_style[fit.band],
            label=fit.band)
ax.legend()

ax.set(xlabel='Incidence Angle (degrees)',
       ylabel='Backscatter (dB)')

# fold

In [ ]:
# doesn't need running again because saved output to: '../results/s1_lake_backscatter.parquet'
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
medians = []
ids = []
for row in tqdm(lakes.to_crs(4326).sample(1).itertuples()):
    
    search = catalog.search(collections=['sentinel-1-rtc'],
                            intersects=row.geometry)
    
    s1items = search.item_collection()
    
    # get most common projection
    vals, cnts = np.unique([item.properties['proj:epsg'] for item in s1items] ,return_counts=True)
    epsg = int(vals[np.argmax(cnts)])
    
    # stack & clip
    ds = stackstac.stack(planetary_computer.sign(s1items),
                        epsg=epsg,
                        bounds_latlon=row.geometry.bounds)
    
    mask = (make_geocube(lakes.to_crs(4326).loc[lakes['id']==row.id],
                          fill=np.nan,
                          like=ds)['id']
             .rename('mask'))
    
    # apply mask and convert to dB
    ds_db = xr.where(mask==row.id,
                     10 * np.log10(ds),
                     np.nan)

    median = (ds_db
              .median(dim=['y','x'], skipna=True)
              .rename('dB'))
    
    medians.append(median) # consider making this a future
    ids.append(row.id)
    
with LocalCluster() as cluster:
    client = cluster.get_client()
    print(client.dashboard_link)    
    computed_medians = dask.compute(*medians)
    dfs = []
    for m_ds, id in zip(computed_medians, ids):
        _df = m_ds.to_dataframe()
        _df['myid'] = id
        dfs.append(_df)
    
    df = pd.concat(dfs).reset_index().set_index('time')
    
df.to_parquet('../results/s1_lake_backscatter.parquet')

using: 
- https://github.com/egagli/generate_sentinel1_local_incidence_angle_maps/blob/882b9940a843c6dc3040f992ad6bed14201a2f5c/generate_lia.py#L61
- https://github.com/microsoft/PlanetaryComputerExamples/blob/main/tutorials/rtc-qualitative-assessment.ipynb
- and sarsen

## get s1 data

In [ ]:
df = pd.read_parquet('../results/s1_lake_backscatter.parquet')
df = df.merge(centrelines['SUBREGION1'], left_on='myid', right_index=True)

In [ ]:
fg = sns.relplot(data=df.groupby(['myid', 'band', 'SUBREGION1'])['dB'].rolling('30d').median().reset_index(),
                 x='time',
                 y='dB',
                 row='SUBREGION1',
                 hue='SUBREGION1',
                 style='myid',
                 palette='tab10',
                 col='band',
                 col_order=['hh'],
                 kind='line',
                 aspect=2,
                #  height=10
                 )

fg.set(ylim=(-22,2))

In [ ]:
df['doy'] = df.index.day_of_year
df['year'] = df.index.year

In [ ]:
smoothed = df.groupby(['myid', 'band', 'SUBREGION1'])['dB'].rolling('30d', min_periods=3, center=True).median().reset_index()
smoothed['doy'] = smoothed['time'].dt.day_of_year
smoothed['year'] = smoothed['time'].dt.year


In [ ]:
sns.relplot(data=smoothed.loc[
    (smoothed['band'].isin(['hh']))
    & (smoothed['year'].isin([2018, 2024]))
    ],
            x='doy',
            y='dB',
            hue='year',
            col='SUBREGION1',
            col_wrap=3,
            # row='SUBREGION1',
            kind='line')